In [146]:
import argparse
import requests

from sys import argv
from bs4 import BeautifulSoup, NavigableString, Tag
from math import ceil 
import string

import pandas as pd

import xml.etree.ElementTree as ET
import re

In [2]:
def load_arguments(parser):
    concepto = vars(parser.parse_args(argv[1:]))["c"]
    return concepto

### Scrapear PubMed

In [3]:
def search_pubmed(concepto):
    
    """Para obtener información de la página PubMed a través del uscador de conceptos. Sin embargo el problema es que sólo obtiene resultados en ingles aunque utilicemos el filtro dle idioma"""

    # URL de origen que se 
    url = "https://pubmed.ncbi.nlm.nih.gov/?term={}".format(concepto)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    num_registros = soup.find('div', class_="results-amount").find('span').text.replace(",", "")

    print("Numero de articulos: {}".format(num_registros))

    num_paginas = ceil(int(num_registros)/10)

    #Bucle generando una lista con todos los articulos
    lista_articulos = list()
    for i in range(1,num_paginas+1):
        url_busqueda = url + "&page={}".format(i)
        page = requests.get(url_busqueda)
        soup = BeautifulSoup(page.content, 'html.parser')

        for articulo in soup.find_all('article', class_="full-docsum"):
            lista_articulos.append(articulo.find('a', href=True, class_='docsum-title')['href'])

    #Bucle en el que se mete en cada articulo y obtiene el abstract
    informacion = list()
    for url_articulo in lista_articulos:
        page = requests.get("https://pubmed.ncbi.nlm.nih.gov/" + url_articulo)
        soup = BeautifulSoup(page.content, 'html.parser')

        try:
            informacion.append(soup.find('div', {"id": "enc-abstract"}).text.replace("\n", " "))

        except AttributeError:
            pass

    print("Numero de abstract obtenidos: {} para la palabra: {}".format(len(informacion), conceto))
    return informacion

In [11]:
def search_pub_esp(concepto):
    
    """Para obtener información de una página muy básica que devulve abstracts de PubMed y Medline en español- El problema está al extraer sólo el texto español del abstract ya que no está muy definido el html"""

    # URL de origen que se 
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
           'AppleWebKit/537.36 (KHTML, like Gecko) '\
           'Chrome/75.0.3770.80 Safari/537.36'}
    
    url = "https://babelmesh.nlm.nih.gov/search/search.php?from=spa&com=&outid=&term={}".format("serologia")
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    for br in soup.findAll('br'):
        next_s = br.nextSibling
        if not (next_s and isinstance(next_s,NavigableString)):
            continue
        next2_s = next_s.nextSibling
        if next2_s and isinstance(next2_s,Tag) and next2_s.name == 'hr':
            text = str(next_s).strip().split()[0]

    print("Numero de articulos: {}".format(text))


    num_paginas = ceil(int(text)/20)

    #Bucle generando una lista con todos los articulos
    lista_articulos = list()
    for i in range(1,num_paginas+1):
        url_busqueda = "https://babelmesh.nlm.nih.gov/search/search.php?submit=Y&page={}&Count=3691&QueryKey=1&WebEnv=MCID_5f7dfb66b4eab545651267fb&from=spa&otype=0&com=".format(i)
        page = requests.get(url_busqueda)
        soup = BeautifulSoup(page.content, 'html.parser')

        for articulo in soup.find_all('p'):
#             print(articulo.find('a', href=True)['href'])
            lista_articulos.append(articulo.find('a', href=True)['href'])
            #print(lista_articulos.append(articulo.find('a', href=True, id= "anch_1")['href']))
#             lista_articulos.append(articulo.find('a', href=True, class_='title')['href'])
            
#         for articulo in soup.find_all('li', class_="document source-encyclopedia-spanish"):
#             lista_articulos.append(articulo.find('a', href=True, class_='title')['href'])
            
#         for articulo in soup.find_all('li', class_="document source-medical-sites-spanish"):
#             lista_articulos.append(articulo.find('a', href=True, class_='title')['href'])

    #Bucle en el que se mete en cada articulo y obtiene el abstract
    informacion = list()
    for url_articulo in lista_articulos:
        page = requests.get("https://babelmesh.nlm.nih.gov/search/" + url_articulo)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        for br in soup.findAll('a'):
            next_s = br.nextSibling
            print(next_s)
#             if not (next_s and isinstance(next_s,NavigableString)):
#                 continue
#             next2_s = next_s.nextSibling
#             if next2_s.name == 'Abstract':
#                 text = str(next_s)
#                 print(text)

#         try:
#             informacion.append(soup.find('div', {"class": "mp-content"}).text.replace("\n", " "))

#         except AttributeError:
#             pass

#     print("Numero de abstract obtenidos: {} para la palabra: {}".format(len(informacion), concepto))
#     return informacion

### Scrappear Medline

In [100]:
def search_medline(concepto):
    
    """Para obtener información de la página Medilne a través del uscador de conceptos. A veces cambia la url del articulo"""

    # URL de origen que se 
    url = "https://vsearch.nlm.nih.gov/vivisimo/cgi-bin/query-meta?v%3Aproject=medlineplus-spanish&v%3Asources=medlineplus-spanish-bundle&query={}".format(concepto)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    num_registros = soup.find('span', class_="intronum").text.replace(",", "")

    print("Numero de articulos: {}".format(num_registros))

    num_paginas = ceil(int(num_registros)/10)

    #Bucle generando una lista con todos los articulos
    lista_articulos = list()
    for i in range(0,num_paginas+1):
        url_busqueda = "https://vsearch.nlm.nih.gov/vivisimo/cgi-bin/query-meta?v%3afile=viv_DSsKHJ&server=pvlbsrch14&v:state=root%7Croot-{}-10%7C0&".format(i*10)
        page = requests.get(url_busqueda)
        soup = BeautifulSoup(page.content, 'html.parser')

        for articulo in soup.find_all('li', class_="document source-mp-directory-spanish"):
            lista_articulos.append(articulo.find('a', href=True, class_='title')['href'])
            
        for articulo in soup.find_all('li', class_="document source-encyclopedia-spanish"):
            lista_articulos.append(articulo.find('a', href=True, class_='title')['href'])
            
        for articulo in soup.find_all('li', class_="document source-medical-sites-spanish"):
            lista_articulos.append(articulo.find('a', href=True, class_='title')['href'])

    #Bucle en el que se mete en cada articulo y obtiene el abstract
    informacion = list()
    for url_articulo in lista_articulos:
        page = requests.get("https://vsearch.nlm.nih.gov/vivisimo/cgi-bin/query-meta?v%3Aproject=medlineplus-spanish&v%3Asources=medlineplus-spanish-bundle/" + url_articulo)
        soup = BeautifulSoup(page.content, 'html.parser')
        print(soup)

        try:
            informacion.append(soup.find('div', {"class": "mp-content"}).text.replace("\n", " "))

        except AttributeError:
            pass

    print("Numero de abstract obtenidos: {} para la palabra: {}".format(len(informacion), concepto))
    return informacion

In [101]:
articulos = search_medline("serologia")

Numero de articulos: 26
Numero de abstract obtenidos: 0 para la palabra: serologia


In [96]:
def search_medline_enciclopedia(concepto):
    
    """Para obtener todos los conceptos y su definición que aparecen en la enciclopedia en español de Medline"""

    # URL de origen que se 
    url = "https://medlineplus.gov/spanish/ency/encyclopedia_{}.htm".format(concepto)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    lista_articulos = list()
    for a in soup.find_all('ul', id="index"):
        for articulo in a.find_all('li'):
            lista_articulos.append(articulo.find('a', href=True)['href'])
            
    #Bucle en el que se mete en cada articulo y obtiene el abstract
    informacion = list()
    for url_articulo in lista_articulos:
        page = requests.get("https://medlineplus.gov/spanish/ency/" + url_articulo)
        soup = BeautifulSoup(page.content, 'html.parser')

        try:
            li1 = soup.find('h1', {"class": "with-also"}).text.replace("\n", " ")
            li2 = soup.find('div', {"class": "main-single"}).find('div', {'id':'ency_summary'}).text.replace("\n", " ")
            li3 = soup.find('div', {"class": "main-single"}).find('div',{'class':'section'}).text.replace("\n", " ")
            li = li1 + " "+ li2 + " " + li3
            informacion.append(li)

        except AttributeError:
            pass

    print("Numero de abstract obtenidos: {} para la palabra: {}".format(len(informacion), concepto))
    return informacion

In [98]:
#Obtenemos todos los conceptos para cada letra del abecedario
enciclopedia = []
alpha = list(string.ascii_uppercase)
for letra in alpha:
    enciclopedia.append(search_medline_enciclopedia(letra))

Numero de abstract obtenidos: 330 para la palabra: A
Numero de abstract obtenidos: 86 para la palabra: B
Numero de abstract obtenidos: 350 para la palabra: C
Numero de abstract obtenidos: 217 para la palabra: D
Numero de abstract obtenidos: 457 para la palabra: E
Numero de abstract obtenidos: 82 para la palabra: F
Numero de abstract obtenidos: 67 para la palabra: G
Numero de abstract obtenidos: 146 para la palabra: H
Numero de abstract obtenidos: 260 para la palabra: I
Numero de abstract obtenidos: 2 para la palabra: J
Numero de abstract obtenidos: 2 para la palabra: K
Numero de abstract obtenidos: 96 para la palabra: L
Numero de abstract obtenidos: 151 para la palabra: M
Numero de abstract obtenidos: 79 para la palabra: N
Numero de abstract obtenidos: 54 para la palabra: O
Numero de abstract obtenidos: 293 para la palabra: P
Numero de abstract obtenidos: 21 para la palabra: Q
Numero de abstract obtenidos: 233 para la palabra: R
Numero de abstract obtenidos: 314 para la palabra: S
Nume

In [99]:
#Pasamos la lista a lista planta y lo guardamos en un csv
flat_enciclop = [item for sublist in enciclopedia for item in sublist]
df = pd.DataFrame(flat_enciclop)
df.to_csv('enciclopedia.csv', index=False)

### Obtener XML de MedlineWeb Service

In [172]:
def search_medline_webservice(concepto):
    
    """Para obtener la información de conceptos mediante Medline Web Service, url que devuelve la información en formato XML"""

    # URL de origen que se
    url = 'https://wsearch.nlm.nih.gov/ws/query?db=healthTopicsSpanish&term={}'.format(concepto)
    response = requests.get(url)
    tree = ET.ElementTree(ET.fromstring(response.text))
    root = tree.getroot()
    docs = root.find('list').findall('document')
    
    lista_articulos = list()
    for doc in docs:
        title = doc.find(".//content[@name='FullSummary']").text.replace("\t", "").replace("\n", "")
        title = re.sub(u'[\<].*?[\>]', '',title)
        lista_articulos.append(title)

    print("Numero de articulos obtenidos: {} para la palabra: {}".format(len(lista_articulos), concepto))
    return lista_articulos

In [177]:
search_medline_webservice("covid")

Numero de articulos obtenidos: 10 para la palabra: covid


['La enfermedad del coronavirus 2019 (COVID-19) es causada por un nuevo virus que se ha expandido por todo el mundo. Se cree que se propaga principalmente a través del contacto cercano de persona a persona. En esta página encontrará enlaces con información sobre síntomas, pruebas, riesgos y cómo puede protegerse usted y su familia.',
 'Las mascotas pueden agregarle diversión, compañía y una sensación de seguridad a su vida. Antes de adquirir una mascota, piense cuidadosamente acerca de qué animal es el mejor para su familia. ¿Qué es lo que cada integrante de la familia busca en una mascota? ¿Quién va a cuidarla? ¿Alguna persona en la casa es alérgica a los animales? ¿Qué tipo de animal se ajusta a su estilo de vida y presupuesto?Cuando ya tenga una mascota, manténgala sana. Aprenda a reconocer los signos de problemas médicos. Lleve a su mascota al veterinario si observa:Pérdida del apetitoConsumo excesivo de aguaAumento o pérdida de peso en forma rápidaComportamiento fuera de lo comúnC